In [1]:
# Load in one of the tables
df1 = spark.sql("select * from default.video_games_5")
df2 = spark.sql("select * from default.home_and_kitchen_5_small")
df3 = spark.sql("select * from default.books_5_small")

#data = df1
#data = df1.union(df2)
data = df1.union(df2).union(df3)

data = data.sample(False, 0.01, seed=0)

data = data.cache()

print((data.count(), len(data.columns)))

(34854, 12)

In [2]:
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer, StopWordsCleaner, Stemmer

from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer, HashingTF, IDF, StringIndexer, SQLTransformer, IndexToString, VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier

# convert text column to spark nlp document
document_assembler = DocumentAssembler() \
    .setInputCol("reviewText") \
    .setOutputCol("document")


# convert document to array of tokens
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
 
# clean tokens 
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

# remove stopwords
stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

# stems tokens to bring it to root form
stemmer = Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")

# Convert custom document structure to array of tokens.
finisher = Finisher() \
    .setInputCols(["stem"]) \
    .setOutputCols(["token_features"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

# Generate Term Frequency
tf = CountVectorizer(inputCol="token_features", outputCol="rawFeatures", vocabSize=10000, minTF=1, minDF=50, maxDF=0.40)

# Generate Inverse Document Frequency weighting
idf = IDF(inputCol="rawFeatures", outputCol="idfFeatures", minDocFreq=5)

# Combine all features into one final "features" column
assembler = VectorAssembler(inputCols=["verified", "overall", "idfFeatures"], outputCol="features")

# Machine Learning Algorithm
#ml_alg  = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.0)
ml_alg  = RandomForestClassifier(numTrees=100, featureSubsetStrategy="auto", impurity='gini', maxDepth=4, maxBins=32)

nlp_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
            finisher,
            tf,
            idf,
            assembler,
            ml_alg])
            

In [3]:
(trainingData, testData) = data.randomSplit([0.8, 0.2], seed = 47)

In [4]:
pipeline_model = nlp_pipeline.fit(trainingData)

In [5]:
predictions =  pipeline_model.transform(testData)
display(predictions)

reviewID overall vote verified reviewTime reviewerID asin reviewerName reviewText summary unixReviewTime label document token normalized cleanTokens stem token_features rawFeatures idfFeatures features rawPrediction probability prediction 443 5.0 0 true 11 7, 2012 AVCM1YMYS7VIT B000006P0J Alan86 This game is a classic, I first played it at the arcade around 1996, but the game came out around 94-95. Of course, you won't find beautiful visuals, this is piece of history, one of the first 3D fighting games ever created. It was groundbreaking for it's time, and to this day I still find it addictive. Each character has a unique fighting style, and these styles keep getting inherited on every sequel, most noticeable Paul, Jack, Yoshimitsu and Law. The Playstation version actually expands the arcade version, adding ending movies and extra characters. There is an arcade version in Tekken 5 as part of the bonus arcade history games, Tekken 2 and 3 are also included. If you enjoyed games like Rival Schools, Mortal Kombat 4 or Dead or Alive, you definately need to play this. The glorious birth of the Tekken franchise. 1352246400 0 List(List(document, 0, 781, This game is a classic, I first played it at the arcade around 1996, but the game came out around 94-95. Of course, you won't find beautiful visuals, this is piece of history, one of the first 3D fighting games ever created. It was groundbreaking for it's time, and to this day I still find it addictive. Each character has a unique fighting style, and these styles keep getting inherited on every sequel, most noticeable Paul, Jack, Yoshimitsu and Law. The Playstation version actually expands the arcade version, adding ending movies and extra characters. There is an arcade version in Tekken 5 as part of the bonus arcade history games, Tekken 2 and 3 are also included. If you enjoyed games like Rival Schools, Mortal Kombat 4 or Dead or Alive, you definately need to play this., Map(sentence -> 0), List())) List(List(token, 0, 3, This, Map(sentence -> 0), List()), List(token, 5, 8, game, Map(sentence -> 0), List()), List(token, 10, 11, is, Map(sentence -> 0), List()), List(token, 13, 13, a, Map(sentence -> 0), List()), List(token, 15, 21, classic, Map(sentence -> 0), List()), List(token, 22, 22, ,, Map(sentence -> 0), List()), List(token, 24, 24, I, Map(sentence -> 0), List()), List(token, 26, 30, first, Map(sentence -> 0), List()), List(token, 32, 37, played, Map(sentence -> 0), List()), List(token, 39, 40, it, Map(sentence -> 0), List()), List(token, 42, 43, at, Map(sentence -> 0), List()), List(token, 45, 47, the, Map(sentence -> 0), List()), List(token, 49, 54, arcade, Map(sentence -> 0), List()), List(token, 56, 61, around, Map(sentence -> 0), List()), List(token, 63, 66, 1996, Map(sentence -> 0), List()), List(token, 67, 67, ,, Map(sentence -> 0), List()), List(token, 69, 71, but, Map(sentence -> 0), List()), List(token, 73, 75, the, Map(sentence -> 0), List()), List(token, 77, 80, game, Map(sentence -> 0), List()), List(token, 82, 85, came, Map(sentence -> 0), List()), List(token, 87, 89, out, Map(sentence -> 0), List()), List(token, 91, 96, around, Map(sentence -> 0), List()), List(token, 98, 102, 94-95, Map(sentence -> 0), List()), List(token, 103, 103, ., Map(sentence -> 0), List()), List(token, 105, 106, Of, Map(sentence -> 0), List()), List(token, 108, 113, course, Map(sentence -> 0), List()), List(token, 114, 114, ,, Map(sentence -> 0), List()), List(token, 116, 118, you, Map(sentence -> 0), List()), List(token, 120, 124, won't, Map(sentence -> 0), List()), List(token, 126, 129, find, Map(sentence -> 0), List()), List(token, 131, 139, beautiful, Map(sentence -> 0), List()), List(token, 141, 147, visuals, Map(sentence -> 0), List()), List(token, 148, 148, ,, Map(sentence -> 0), List()), List(token, 150, 153, this, Map(sentence -> 0), List()), List(token, 155, 156, is, Map(sentence -> 0), List()), List(token, 158, 162, piece, Map(sentence -> 0), List()), List(token, 164, 165, of,

In [6]:
predictions.groupBy("label").count().show()
predictions.groupBy("prediction").count().show()

+-----+-----+
label|count|
+-----+-----+
 1| 1289|
 0| 5816|
+-----+-----+

+----------+-----+
prediction|count|
+----------+-----+
 0.0| 7092|
 1.0| 13|
+----------+-----+

In [7]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
pre_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
rec_evaluator = MulticlassClassificationEvaluator(metricName="weightedRecall")
pr_evaluator  = BinaryClassificationEvaluator(metricName="areaUnderPR")
auc_evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")

print("Test Accuracy       = %g" % (acc_evaluator.evaluate(predictions)))
print("Test Precision      = %g" % (pre_evaluator.evaluate(predictions)))
print("Test Recall         = %g" % (rec_evaluator.evaluate(predictions)))
print("Test areaUnderPR    = %g" % (pr_evaluator.evaluate(predictions)))
print("Test areaUnderROC   = %g" % (auc_evaluator.evaluate(predictions)))

Test Accuracy = 0.819564
Test Precision = 0.810508
Test Recall = 0.819564
Test areaUnderPR = 0.459839
Test areaUnderROC = 0.800264